In [1]:
import time
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim
from gensim import corpora, models, parsing
from gensim.models import LdaModel
import nltk
import pickle
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import *
import numpy as np
from sklearn.neighbors import NearestNeighbors
import spacy
from spacy.lang.en import English
import pyLDAvis.gensim

In [2]:
parser = English()
nltk.download('wordnet', quiet=True)
nltk.download('stopwords', quiet=True)
np.random.seed(2018)
en_stop = set(nltk.corpus.stopwords.words('english'))

In [3]:
class PandasHelper:

    @staticmethod
    def get_id_from_series(series):
        return int(series.index.tolist()[0])

In [4]:
class LDAModel:
    def __init__(self):
        # Num_topics in LDA
        self.num_topics = 20
        # Filter out tokens that appear in less than `no_below`
        self.no_below = 5
        # Number of iterations in training LDA model, the less the documents in total, the more the iterations for LDA model to converge
        self.num_of_iterations = 1000

    def _lemmatize(self, text):
        return WordNetLemmatizer().lemmatize(text)
    
    def _tokenize(self, text):
        lda_tokens = []
        tokens = parser(text)
        for token in tokens:
            if token.orth_.isspace():
                continue
            else:
                lda_tokens.append(token.lower_)
        return lda_tokens
    
    def _preprocess(self, text):
        tokens = self._tokenize(text)
        tokens = [token for token in tokens if len(token) > 4]
        tokens = [token for token in tokens if token not in en_stop]
        tokens = [self._lemmatize(token) for token in tokens]
        return tokens
    
    def _savePickleFile(self, fileName, data):
        fileName = f'./{fileName}.pickle'
        mappingFile = open(fileName, 'wb')
        pickle.dump(data, mappingFile)
        mappingFile.close()

    def saveModel(self, lda, corpus, docs_topics):
        # Save model output
        lda.save('./model')
        # Save corpus
        self._savePickleFile('corpus', corpus)
        self._savePickleFile('docs_topics', docs_topics)

    def trainModel(self):
        data = pd.read_json('../data/movies_data.json', orient='split')
        documents = data['content']
        ids = data['id']
        processed_docs = documents.map(self._preprocess)

        print('Start training LDA model...')
        dictionary = gensim.corpora.Dictionary(processed_docs)
        print(len(dictionary))
        dictionary.filter_extremes(no_below = self.no_below)
        corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
        
        lda = LdaModel(
            corpus,
            num_topics=self.num_topics,
            id2word=dictionary,
            passes=5,
            alpha=[0.01]*self.num_topics,
            iterations=self.num_of_iterations,
            minimum_probability=0.0)
        
        # lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
        # pyLDAvis.show(lda_display)
        
        docs_topics = np.array([[tup[1] for tup in lst] for lst in lda[corpus]])
        df_docs_topics = pd.DataFrame(docs_topics, index = ids)
        
        print('Finished training LDA model...')
        
        return lda, corpus, df_docs_topics
    
lda_model = LDAModel()
lda, corpus, df_docs_topics = lda_model.trainModel()  # train a LDA model using the assgined corpora
lda_model.saveModel(lda, corpus, df_docs_topics) # save model for recommendations use

Start training LDA model...
31674
Finished training LDA model...


In [5]:
class Recommender():
    def __init__(self):
        self.lda = LdaModel.load('./model')
        self.corpus = self.loadPickleFile('./corpus')
        self.docs_topics = self.loadPickleFile('./docs_topics')
        self.num_of_recommendation = 10
        
    def loadPickleFile(self, fileName):
        file = open(f'{fileName}.pickle','rb')
        object_file = pickle.load(file)
        return object_file
    
    def recommend(self, movie_id):
        data = pd.read_json('../data/movies_data.json', orient='split')
        start = time.time()

        model_knn = NearestNeighbors(metric='cosine', algorithm='brute')
        model_knn.fit(self.docs_topics)

        movie_topics = self.get_movie_topics(movie_id)

        if movie_topics is None:
            return None

        distances, indices = model_knn.kneighbors(movie_topics, n_neighbors=self.num_of_recommendation + 1)
        similarities = 1 - distances.flatten()
        similarities = similarities[1:]
        indices = indices.flatten()
        indices = indices[1:]

        end = time.time()
        print(f'Recommended in: {end - start} s')
        return [{
            'id': PandasHelper.get_id_from_series(self.docs_topics.iloc[[indices[index]]]),
            'name': data[data['id'] == PandasHelper.get_id_from_series(self.docs_topics.iloc[[indices[index]]])].title.iloc[0],
            'similarity': float(line)
        } for index, line in enumerate(similarities)]

    def get_movie_topics(self, movie_id):
        movie_row = self.docs_topics[self.docs_topics.index == movie_id]

        if movie_row.empty:
            return None

        row_values = movie_row.values.reshape(1, -1)

        return row_values

In [6]:
recommender = Recommender()

In [7]:
recommender.recommend(480)

Recommended in: 0.003949880599975586 s


[{'id': 1193,
  'name': "One Flew Over the Cuckoo's Nest",
  'similarity': 0.9909398555755615},
 {'id': 6, 'name': 'Heat', 'similarity': 0.9894849061965942},
 {'id': 1284, 'name': 'The Big Sleep', 'similarity': 0.9867210984230042},
 {'id': 74327,
  'name': 'The First Day of the Rest of Your Life',
  'similarity': 0.9867028594017029},
 {'id': 1914, 'name': 'Smoke Signals', 'similarity': 0.9854169487953186},
 {'id': 2033, 'name': 'The Black Cauldron', 'similarity': 0.984953761100769},
 {'id': 848, 'name': 'The Spitfire Grill', 'similarity': 0.98486328125},
 {'id': 1332, 'name': 'The Believers', 'similarity': 0.98486328125},
 {'id': 2006, 'name': 'The Mask of Zorro', 'similarity': 0.9848094582557678},
 {'id': 266, 'name': 'Legends of the Fall', 'similarity': 0.9847936034202576}]